In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datetime
from gee_scripts.get_sources import get_explanatory_composite
from gee_scripts.parameters import explain_vars
from gee_scripts.get_sources import get_s1_dates
from gee_scripts.gee import export_classifier
from gee_scripts.directories import get_export_folder
from gee_scripts import init_ee

init_ee()
import ee


## 1. Train the classifier in GEE and save the model as asset

In [ ]:
# This table contains the training data for the model
training_data = ee.FeatureCollection("projects/ee-marortpab/assets/FAO/indonesia/gwl/df_with_models")

In [ ]:
# We have intentified three potential models: model5, model7, and model17
# export_classifier(training_data, "model7")

# I have already processed and exported the models, the function above
# can be used to export different models.
# Check the function `export_classifier` for more details.

ee_model_17 = "projects/sepal-ui-421413/assets/gwl-modeling/models/RandomForest_model17"
ee_model_5 = "projects/sepal-ui-421413/assets/gwl-modeling/models/RandomForest_model5"
ee_model_7 = "projects/sepal-ui-421413/assets/gwl-modeling/models/RandomForest_model7"

## 2. Estimate using the trained model

https://code.earthengine.google.com/1fd31fe53d5a8cf8b812552f901325a0

## 2.1. Set parameters

In [ ]:
# This lie will take some time to run, because it will do a GEE request
all_s1_dates = get_s1_dates(aoi=target_phu)

In [ ]:
all_str_date = [
    datetime.datetime.fromtimestamp(date/1000).strftime('%Y-%m-%d') for date in all_s1_dates
]

In [ ]:
ee.Date('2015-01-02')

In [ ]:
def export_estimation(model_name, aoi_name, str_date, target_phu, ee_classifier, ee_aoi):
    """Export the estimated GWL image for a given model and target date."""

    # Get explanatory composite closest to target date
    image = get_explanatory_composite(
        target_date=str_date, 
        ee_region=target_phu, 
        max_days_offset=1
    ).select(explain_vars)

    output_image_name = f"{model_name}_{aoi_name}_{str_date}"
    estimated_image = image.select(explain_vars).classify(ee_classifier).set({"model": model_name, "date": str_date})

    # create export task
    task = ee.batch.Export.image.toAsset(
        **{
            "image": estimated_image,
            "description": output_image_name,
            "assetId": str(get_export_folder(output_folder=f"estimation/{model_name}")/output_image_name),
            "region": ee_aoi,
            "scale": 100,
        }
    )
    return task

In [ ]:
model_name = "model7"
ee_aoi = phus.filter(ee.Filter.eq("PHU_NAME", "KHG Sungai Siak - Sungai Kampar")).geometry()
aoi_name = "PHU_SungaiKampar"
ee_classifier = ee_model_7
phus = ee.FeatureCollection("users/marortpab/FAO/SEPAL/2023_trainings/smm/AOI__Province__865_PHUs__INDONESIA")

tasks = [export_estimation(model_name, target_date, target_phu, ee_classifier) for target_date in all_str_date[:10]]

In [ ]:
[task.start() for task in tasks]